In [30]:
import torch
from torch import nn
from torch.nn import functional as F

In [31]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
X = torch.rand(2, 20)
net(X).shape # 其实是 net.__call__(X).shape 的缩写

torch.Size([2, 10])

In [32]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class nn.Module to perform 调用父类 nn.Mudule 的构造函数
        # the necessary initialization 进行必要的初始化
        super().__init__() # 调用父类
        self.hidden = nn.LazyLinear(256)  # 隐藏层，输出特征数为 256
        self.out = nn.LazyLinear(10)      # 输出层，输出特征数为 10

    # Define the forward propagation of the model, that is, how to return the 定义模型的前向传播
    # required model output based on the input X 即如何基于输入 X 返回所需的模型输出
    def forward(self, X):
        return self.out(F.relu(self.hidden(X))) # F.relu 是 PyTorch 中的激活函数，用于将所有负值置为 0

In [33]:
net = MLP()
net(X).shape

torch.Size([2, 10])

In [34]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)
            # 使用 nn.Module 类中的 add_module 方法将每个模块添加进来

    def forward(self, X):
        for module in self.children():
            # self.children() 是 PyTorch 的一个方法，它返回当前模块（在这里是 MySequential 实例）中所有直接子模块的迭代器。
            # for module in self.children(): 会遍历所有子模块，并按顺序对每个子模块执行前向传播。
            X = module(X)
        return X

In [35]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
print(net)
net(X).shape

MySequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): LazyLinear(in_features=0, out_features=10, bias=True)
)


torch.Size([2, 10])

In [41]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 随机权重参数，在训练过程中不会计算梯度并保持不变
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X) # 这里就已经用内置的一个方法初始化并赋值了线性变换的 W 和 b
        X = F.relu(X @ self.rand_weight + 1) # 这个自定义的 FixedHiddenMLP 只是想展示一下如何使用常量
        X = self.linear(X) # 重用全连接层。这相当于与两个全连接层共享参数（ w 和 b ）
        while X.abs().sum() > 1: # 控制流
            X /= 2
        return X.sum()

In [42]:
net = FixedHiddenMLP()
print(net)
net(X)

FixedHiddenMLP(
  (linear): LazyLinear(in_features=0, out_features=20, bias=True)
)


tensor(0.0276, grad_fn=<SumBackward0>)

In [43]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(-0.1952, grad_fn=<SumBackward0>)

In [48]:
"""Exercise
1. 如果将模块存储在 Python 列表中而不是使用 nn.ModuleList，则这些模块不会被 nn.Module 类检测到，因此它们的参数不会被注册到模型中。结果是，当调用 model.parameters() 或 model.children() 等方法时，Python 列表中的模块参数不会被包含进去，这将导致这些模块的参数无法参与训练和自动微分过程

2. """
class ParallelModule(nn.Module):
    def __init__(self, net1, net2):
        super().__init__()
        self.net1 = net1
        self.net2 = net2

    def forward(self, X):
        out1 = self.net1(X)
        out2 = self.net2(X)
        return torch.cat((out1, out2), dim=1) 
"""
3. """
def create_network(module_class, num_instances, *args, **kwargs):
    modules = [module_class(*args, **kwargs) for _ in range(num_instances)]
    return nn.Sequential(*modules)